In [12]:
import sys
import os
sys.path.append(os.path.abspath('../../'))

In [13]:
import pandas as pd

from mcda.electre.concordance import (
    concordance_with_interactions,
    concordance_with_interactions_marginal,
    FunctionType,
    Interaction,
    InteractionType
)
from mcda.core.scales import QuantitativeScale, PreferenceDirection
from mcda.core.functions import Threshold

1. Create dataframes

In [14]:
alternative_names = ['fiat', 'vwPolo' , 'nissan' , 'toyota' , 'suzuki', 'ford' ]
criteria_names = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
profile_names = ['P1', 'P2', 'P3']

In [15]:
cars = pd.DataFrame(
    [
        [4000, 311, 0.988, 90, 2, 8.5],
        [10000, 150, 0.8, 60, 4, 6.66],
        [21370, 255, 0.5, 90, 5, 7.77],
        [9999, 99, 0.9, 99, 9, 9.99],
        [12345, 12, 0.76, 43, 4, 1.5],
        [0, 1, 0, 0, 0, 0],
    ],
    index=alternative_names,
    columns=criteria_names,
)
cars

,C1,C2,C3,C4,C5,C6
fiat,4000,311,0.988,90,2,8.50
vwPolo,10000,150,0.800,60,4,6.66
nissan,21370,255,0.500,90,5,7.77
toyota,9999,99,0.900,99,9,9.99
suzuki,12345,12,0.760,43,4,1.50
ford,0,1,0.000,0,0,0.00


In [16]:
profiles = pd.DataFrame(
    [
        [1200, 25, 0.3, 40, 9, 7.0],
        [3500, 45, 0.6, 60, 4, 6.7],
        [3400, 90, 0.7654, 98, 2, 8.8]
    ],
    index=profile_names,
    columns=criteria_names
) 
profiles

,C1,C2,C3,C4,C5,C6
P1,1200,25,0.3000,40,9,7.0
P2,3500,45,0.6000,60,4,6.7
P3,3400,90,0.7654,98,2,8.8


2. Create series with thresholds and specify criteria scales and weights

In [17]:
scales = pd.Series([
    QuantitativeScale(0, 50000),
    QuantitativeScale(0, 1000),
    QuantitativeScale(0.0, 1.0),
    QuantitativeScale(0.0, 150.0),
    QuantitativeScale(0.0, 15.0, PreferenceDirection.MIN),
    QuantitativeScale(0.0, 15.0, PreferenceDirection.MIN),
], index=criteria_names)
scales

C1      Scale [0, 50000], max direction
C2       Scale [0, 1000], max direction
C3      Scale [0.0, 1.0], max direction
C4    Scale [0.0, 150.0], max direction
C5     Scale [0.0, 15.0], min direction
C6     Scale [0.0, 15.0], min direction
dtype: object

In [18]:
weights = pd.Series([2, 3, 5, 1, 3, 4], index=criteria_names)
weights

C1    2
C2    3
C3    5
C4    1
C5    3
C6    4
dtype: int64

In [19]:
indifference = pd.Series([
    Threshold(0.1, 1000),
    Threshold(0, 500),
    Threshold(0, 0.1),
    Threshold(0.1, 0),
    Threshold(0.2, 2),
    Threshold(0, 1.5),
], index=criteria_names)
indifference

C1    Threshold: alpha=0.1, beta=1000
C2       Threshold: alpha=0, beta=500
C3       Threshold: alpha=0, beta=0.1
C4       Threshold: alpha=0.1, beta=0
C5       Threshold: alpha=0.2, beta=2
C6       Threshold: alpha=0, beta=1.5
dtype: object

In [20]:
preference = pd.Series([
    Threshold(0.15, 1500),
    Threshold(0, 550),
    Threshold(0, 0.4),
    Threshold(0.4, 0.2),
    Threshold(0.2, 10),
    Threshold(0, 4.5),
], index=criteria_names)
preference

C1    Threshold: alpha=0.15, beta=1500
C2        Threshold: alpha=0, beta=550
C3        Threshold: alpha=0, beta=0.4
C4      Threshold: alpha=0.4, beta=0.2
C5       Threshold: alpha=0.2, beta=10
C6        Threshold: alpha=0, beta=4.5
dtype: object

In [21]:
interactions = pd.DataFrame([
    [None, None, None, None, None, None],
    [None, None, None, None, None, None],
    [Interaction(InteractionType.A, 1.4), None, None, None, None, None],
    [None, None, None, None, None, None],
    [None, None, None, None, None, Interaction(InteractionType.MS, 3.5)],
    [None, Interaction(InteractionType.MW, -0.6), None, None, None, None],
], index=criteria_names, columns=criteria_names)
interactions

,C1,C2,C3,C4,C5,C6
C1,None,None,None,None,None,None
C2,None,None,None,None,None,None
C3,Antagonistic | 1.4,None,None,None,None,None
C4,None,None,None,None,None,None
C5,None,None,None,None,None,Mutual Strengthening | 3.5
C6,None,Mutual Weakening | -0.6,None,None,None,None


3. Short example with comprehensive index calculations

Between fiat and suzuki:

In [22]:
concordance_with_interactions_marginal(
    cars.loc['fiat'],
    cars.loc['suzuki'],
    scales,
    weights,
    indifference,
    preference,
    interactions
)

0.6385542168674698

5. Concordance table between alternatives and alternatives, with min function to capture the interactions effect

In [23]:
concordance_with_interactions(
    cars,
    scales,
    weights,
    indifference,
    preference,
    interactions,
)

,fiat,vwPolo,nissan,toyota,suzuki,ford
fiat,1.000000,0.872031,0.897436,0.897436,0.638554,0.777778
vwPolo,0.881978,1.000000,0.846154,0.948718,0.717470,0.752778
nissan,0.760766,0.840510,1.000000,0.760766,0.629630,0.736111
toyota,0.938462,0.837009,0.838598,1.000000,0.690227,0.669444
suzuki,0.850080,0.951485,0.846154,0.916602,1.000000,0.977914
ford,0.617225,0.617225,0.617225,0.617225,0.617225,1.000000


6. Concordance table between alternatives and profiles, with multiply function to capture the interactions effect.

In [24]:
alt_prof, prof_alt = concordance_with_interactions(
    cars,
    scales,
    weights,
    indifference,
    preference,
    interactions,
    FunctionType.MUL,
    profiles,
)

In [25]:
alt_prof

,P1,P2,P3
fiat,1.000000,0.979178,1.000000
vwPolo,1.000000,1.000000,0.952153
nissan,1.000000,1.000000,0.868102
toyota,0.897908,0.850370,0.938462
suzuki,1.000000,0.953614,0.952153
ford,0.748777,0.617225,0.617225


In [26]:
prof_alt

,fiat,vwPolo,nissan,toyota,suzuki,ford
P1,0.528205,0.585276,0.761476,0.617225,0.363889,0.653263
P2,0.721742,0.816361,0.846154,0.690049,0.585308,0.752778
P3,0.898200,0.848881,0.897436,0.868948,0.638554,0.777778
